You can see the official tutorial [here](https://github.com/catboost/tutorials/blob/master/text_features/text_features_in_catboost.ipynb).

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score


def label_encoding(train: pd.DataFrame, test: pd.DataFrame, col_definition: dict):
    """
    col_definition: encode_col
    """
    n_train = len(train)
    train = pd.concat([train, test], sort=False).reset_index(drop=True)
    for f in col_definition['encode_col']:
        try:
            lbl = preprocessing.LabelEncoder()
            train[f] = lbl.fit_transform(list(train[f].values))
        except:
            print(f)
    test = train[n_train:].reset_index(drop=True)
    train = train[:n_train]
    return train, test

In [2]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')
sub = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
target_col = 'target'
text_cols = ['text']
categorical_cols = ['keyword', 'location']

In [5]:
train, test = label_encoding(train, test, col_definition={'encode_col': categorical_cols})

In [6]:
X_train = train[text_cols + categorical_cols]
y_train = train[target_col].values
X_test = test[text_cols + categorical_cols]

In [7]:
y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

catboost_params = {
    'iterations': 1000,
    'learning_rate': 0.1,
    'eval_metric': 'Logloss',
    'task_type': 'GPU',
    'early_stopping_rounds': 10,
    'use_best_model': True,
    'verbose': 100
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    train_pool = Pool(
        X_tr, 
        y_tr, 
        cat_features=categorical_cols,
        text_features=text_cols,
        feature_names=list(X_tr)
    )
    valid_pool = Pool(
        X_val, 
        y_val, 
        cat_features=categorical_cols,
        text_features=text_cols,
        feature_names=list(X_tr)
    )

    model = CatBoostClassifier(**catboost_params)
    model.fit(train_pool, eval_set=valid_pool)

    oof_train[valid_index] = model.predict_proba(X_val)[:, 1]

    y_pred = model.predict_proba(X_test)[:, 1]
    y_preds.append(y_pred)
    models.append(model)

0:	learn: 0.6532298	test: 0.6486294	best: 0.6486294 (0)	total: 58.4ms	remaining: 58.3s
100:	learn: 0.4464737	test: 0.4256285	best: 0.4256285 (100)	total: 4.54s	remaining: 40.4s
bestTest = 0.4226969508
bestIteration = 134
Shrink model to first 135 iterations.
0:	learn: 0.6497926	test: 0.6478401	best: 0.6478401 (0)	total: 91.7ms	remaining: 1m 31s
bestTest = 0.4550229008
bestIteration = 47
Shrink model to first 48 iterations.
0:	learn: 0.6506466	test: 0.6462198	best: 0.6462198 (0)	total: 48.6ms	remaining: 48.6s
bestTest = 0.443431992
bestIteration = 54
Shrink model to first 55 iterations.
0:	learn: 0.6522775	test: 0.6489454	best: 0.6489454 (0)	total: 75.3ms	remaining: 1m 15s
bestTest = 0.4413913321
bestIteration = 49
Shrink model to first 50 iterations.
0:	learn: 0.6489978	test: 0.6474633	best: 0.6474633 (0)	total: 35.7ms	remaining: 35.6s
bestTest = 0.4652266032
bestIteration = 77
Shrink model to first 78 iterations.


In [8]:
pd.DataFrame(oof_train).to_csv('oof_train_skfold.csv', index=False)
print(f'Local AUC: {roc_auc_score(y_train, oof_train)}')
print(f'Local ACC: {accuracy_score(y_train, (oof_train > 0.5).astype(int))}')

Local AUC: 0.8590278934640654
Local ACC: 0.7985025614081177


In [9]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)
y_sub[:10]

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [10]:
sub[target_col] = y_sub
sub.to_csv('submission.csv', index=False)
sub.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
